In [ ]:
#| default_exp pds.ctx_index

# CTX Index

> Scraping the latest CTX index from the latest added volume.

The CTX Index is special as it's latest URL is dynamic, so it needs to be web-scraped.

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
from dataclasses import dataclass
from string import Template

import pandas as pd


In [ ]:
#| export
@dataclass
class CTXIndex:
    """Class to determine the URL for the latest cumulative index.

    This is a 2 step process, where first the MRO release page is scraped
    for the latest CTX release, and then the latest release page is scraped
    for the latest volume.
    From that latest volume the latest index URL is constructed.
    """
    volumes_url: str = "https://pds-imaging.jpl.nasa.gov/volumes/mro.html"
    release_url_template: Template = Template(
        "https://pds-imaging.jpl.nasa.gov/volumes/mro/release${release}.html"
    )
    volume_url_template: Template = Template(
        "https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_${volume}/"
    )
    scraped_tables: bool = False
    release_scraped: bool = False

    @property
    def web_tables_list(self):
        """Use the pandas scraper to read in the MRO data release table.

        The scraper returns several tables in a list and the last one
        lists all the CTX volumes.

        This could be replaced by cached properties.
        """
        if not self.scraped_tables:
            self._list_of_scraped_tables = pd.read_html(self.volumes_url)
            self.scraped_tables = True
        return self._list_of_scraped_tables

    @property
    def release_number(self):
        """Fishes out the release number.

        This is needed to construct the exact URL to the latest cumulative index file.
        """
        alist = self.web_tables_list
        return alist[-1].iloc[-1, 0].split()[-1]

    @property
    def release_url(self):
        "Constructs the release URL from the release number."
        return self.release_url_template.substitute(release=self.release_number)

    @property
    def latest_volume_url(self):
        """Scrape the Release URL for the latest volume URL in that.

        This is necessary because a release usually has more that one volume.
        """
        if not self.release_scraped:
            alist = pd.read_html(self.release_url)
            # get last row of 4th table
            row = alist[3].iloc[-1]
            number = None
            # first number that is NAN breaks the loop over last row of table
            for elem in row.values:
                try:
                    number = int(elem.split()[-1])
                except AttributeError:
                    break
            self.number = number
            self.release_scraped = True
        return self.volume_url_template.substitute(volume=self.number)

    @property
    def latest_index_label_url(self):
        "Construct the URL for the latest cumulative index."

        return self.latest_volume_url + "index/cumindex.lbl"

In [ ]:
ctx = CTXIndex()

In [ ]:
ctx.latest_index_label_url

'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_4191/index/cumindex.lbl'

In [ ]:
show_doc(CTXIndex.web_tables_list)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/pds/ctx_index.py#L33){target="_blank" style="float:right; font-size:smaller"}

### CTXIndex.web_tables_list

>      CTXIndex.web_tables_list ()

Use the pandas scraper to read in the MRO data release table.

The scraper returns several tables in a list and the last one
lists all the CTX volumes.

This could be replaced by cached properties.

In [ ]:
ctx.web_tables_list[-1].tail()

,Mission to MarsContext Camera (CTX) and Mars Color Imager (MARCI) MRO LEVEL 0 EDRs V1.0,Mission to MarsContext Camera (CTX) and Mars Color Imager (MARCI) MRO LEVEL 0 EDRs V1.0.1,Mission to MarsContext Camera (CTX) and Mars Color Imager (MARCI) MRO LEVEL 0 EDRs V1.0.2
57,Release 58,NaN,CTX (86 volumes mrox_3959-4044) and MARCI (28 ...
58,Release 59,NaN,CTX (54 volumes mrox_4045-4098) and MARCI (19 ...
59,Release 60,NaN,CTX (35 volumes mrox_4099-4133) and MARCI (12 ...
60,Release 61,NaN,CTX (29 volumes mrox_4134-4162) and MARCI (12 ...
61,Release 62,NaN,CTX (29 volumes mrox_4163-4191) and MARCI (10 ...


In [ ]:
assert type(ctx.web_tables_list) == list

In [ ]:
assert len(ctx.web_tables_list) == 4

In [ ]:
show_doc(CTXIndex.release_number)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/pds/ctx_index.py#L47){target="_blank" style="float:right; font-size:smaller"}

### CTXIndex.release_number

>      CTXIndex.release_number ()

Fishes out the release number.

This is needed to construct the exact URL to the latest cumulative index file.

In [ ]:
assert type(ctx.release_number) == str
ctx.release_number

'62'

In [ ]:
show_doc(CTXIndex.release_url)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/pds/ctx_index.py#L56){target="_blank" style="float:right; font-size:smaller"}

### CTXIndex.release_url

>      CTXIndex.release_url ()

Constructs the release URL from the release number.

In [ ]:
ctx.release_url

'https://pds-imaging.jpl.nasa.gov/volumes/mro/release62.html'

In [ ]:
assert type(ctx.release_url) == str

In [ ]:
show_doc(CTXIndex.latest_volume_url)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/pds/ctx_index.py#L61){target="_blank" style="float:right; font-size:smaller"}

### CTXIndex.latest_volume_url

>      CTXIndex.latest_volume_url ()

Scrape the Release URL for the latest volume URL in that.

This is necessary because a release usually has more that one volume.

In [ ]:
ctx.latest_volume_url

'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_4191/'

In [ ]:
assert type(ctx.latest_volume_url) == str

In [ ]:
show_doc(CTXIndex.latest_index_label_url)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/pds/ctx_index.py#L82){target="_blank" style="float:right; font-size:smaller"}

### CTXIndex.latest_index_label_url

>      CTXIndex.latest_index_label_url ()

Construct the URL for the latest cumulative index.

In [ ]:
ctx.latest_index_label_url

'https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_4191/index/cumindex.lbl'